### Tokenizers

In [1]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=6, linewidth=75, sci_mode=False)

tokenizer是NLP管道的核心组件之一，作用只有一个：将文本转换为模型可以处理的数据。模型只能处理数字，因此tokenizer需要将文本输入转换为数字。在本节中，我们将探索标记化管道中究竟发生了什么。

在NLP任务中，通常处理的数据是原始文本。下面是这样一个文本的例子: Jim Henson was a puppeteer

然而，模型只能处理数字，因此我们需要找到一种将原始文本转换为数字的方法。这就是tokenizer所做的，有很多方法可以做到这一点。目标是找到最有意义的表示——也就是对模型最有意义的表示——如果可能的话，找到最小的表示。

让我们看一些标记化算法的示例，并尝试回答关于标记化的一些问题。

#### Word-based

首先想到的标记器是基于单词的。这非常容易设置和使用，只需要一些规则，并且通常会产生不错的结果。例如，在下图中，目标是将原始文本分成单词，并为每个单词找到一个数字表示:
![img](./img/1.png)

分割文本有不同的方法。例如，我们可以通过应用Python的split()函数使用空格将文本标记为单词:

In [2]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


还有一些不同的单词标记器，它们有额外的标点符合规则。使用这种标记器，我们最终可以得到一些相当大的“词汇表”，其中词汇表由语料库中独立标记的总数定义。

每个单词被分配一个ID，从0开始，一直到词汇表的大小。模型使用这些id来识别每个单词。

如果我们想用基于单词的标记器完全覆盖一种语言，我们需要为语言中的每个单词提供一个标识符，这将生成大量的标记。例如，英语中有超过500,000个单词，因此要构建从每个单词到输入ID的映射，我们需要跟踪这么多ID。此外，像“dog”这样的词与“dogs”这样的词的表示方式不同，模型最初将无法知道“dog”和“dogs”是相似的：它会将这两个词识别为不相关的。这同样适用于其他类似的单词，如“run”和“running”，模型最初不会认为它们相似。

最后，我们需要一个自定义标记来表示词汇表中没有的单词。这被称为“unknown”令牌，通常表示为“[UNK]”或“”。如果您看到标记器产生了很多这样的标记，这通常是一个不好的迹象，因为它无法检索一个词的合理表示，并且正在丢失信息。制作词汇表时的目标是使标记器将尽可能少的单词标记为未知标记。

减少未知标记数量的一种方法是更深入一层，使用character-based的标记器。

#### Character-based

基于字符的标记器将文本分割成字符，而不是单词。这有两个主要好处:
- 词汇量要小得多
- 这里有更少的词汇表外(unknown)标记，因为每个单词都可以由字符构建。

但是这里也出现了一些关于空格和标点的问题:
![img](./img/2.png)

这种方法也不是完美的。由于现在的表示是基于字符而不是单词，有人可能会说，从直觉上讲，它的意义更小:每个字符本身并不意味着很多，而单词就是这种情况。然而，这又因语言而异;例如，在汉语中，每个字符比拉丁语言中的一个字符携带更多的信息。

另一件需要考虑的事情是，我们的模型最终会处理巨量的令牌:使用基于单词的令牌器，一个单词可能只是一个令牌，但在转换为字符时，它很容易变成10个或更多的令牌。

为了两全其美，我们可以使用结合了这两种方法的第三种技术: subword tokenization。

####  subword tokenization

子词分词算法依赖的原则是，频繁使用的词不应该被分解成更小的子词，而不常用的词应该被分解成有意义的子词。

例如，“annoying”可能被认为是一个罕见的词，可以分解为“annoying”和“ly”。这两个词可能会更频繁地作为独立的子词出现，而与此同时，“annoying”的意思被“annoying”和“ly”的复合意思所保留。

下面是一个示例，展示了子词标记化算法如何对序列“Let 's do tokenization!”：
![img](./img/3.png)

这些子词最终提供了大量的语义含义: 例如，在上面的示例中，“tokenization”被拆分为“token”和“ization”，这两个令牌具有语义含义，同时又节省空间(只需要两个令牌来表示长单词)。这使我们能够相对较好地覆盖较小的词汇表，并且几乎没有未知的标记。

这种方法在像土耳其语这样的粘连语言中特别有用，在这种语言中，您可以通过将子词串在一起形成(几乎)任意长的复杂单词。

#### And more!

当前还有更多的相关技术，举几个例子:
- Byte-level BPE, as used in GPT-2
- WordPiece, as used in BERT
- SentencePiece or Unigram, as used in several multilingual models

现在，您应该对标记器的工作方式有了足够的了解，从而可以开始使用API。

#### Loading and saving

加载和保存标记器与使用模型一样简单。实际上，它基于相同的两个方法: from_pretrained()和save_pretrained()。这些方法将加载或保存标记器使用的算法(有点像模型的框架)以及它的词汇表(有点像模型的权重)。

加载与BERT相同checkpoint训练的BERT标记器的方式与加载模型的方式相同，只是我们使用了BertTokenizer类:

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

与AutoModel类似，AutoTokenizer类将根据checkpoint名称在库中获取适当的标记器类，并且可以直接与任何检查点一起使用:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

我们现在可以像上一节所示的那样使用标记器:

In [5]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

保存标记器与保存模型相同:

In [7]:
tokenizer.save_pretrained("./tokenizers")

('./tokenizers\\tokenizer_config.json',
 './tokenizers\\special_tokens_map.json',
 './tokenizers\\vocab.txt',
 './tokenizers\\added_tokens.json',
 './tokenizers\\tokenizer.json')

我们将在第3章中更多地讨论token_type_ids，稍后我们将解释attention_mask键。首先，让我们看看input_ids是如何生成的。为此，我们需要查看标记器的中间方法。

#### Encoding

将文本转换为数字称为编码。编码分两步完成:标记化，然后转换为输入id。

正如我们所看到的，第一步是将文本分成单词(或单词的一部分、标点符号等)，通常称为标记。有多个规则可以控制该过程，这就是为什么我们需要使用模型的名称来实例化标记器，以确保我们使用的规则与模型预训练时使用的规则相同。

第二步是将这些符号转换为数字，这样我们就可以用它们构建一个tensor，并将它们提供给模型。为此，标记器有一个词汇表，这是我们使用from_pretrained()方法实例化它时下载的部分。同样，我们需要使用与模型预训练时相同的词汇表。

为了更好地理解这两个步骤，我们将分别研究它们。请注意，我们将使用一些方法分别执行标记化管道的各个部分，以向您展示这些步骤的中间结果，但在实践中，应该在输入上直接调用tokenizer(如第2节所示)。

#### Tokenization

标记化过程由标记器的tokenize()方法完成:

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
tokens

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']

这个标记器是一个子词标记器: 它将单词拆分，直到获得可以由其词汇表表示的标记。这里的transformer就是这种情况，它被分成两个令牌:transform和##er。

#### From tokens to input IDs

input IDs的转换由convert_tokens_to_ids()标记器方法处理:

In [12]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[7993, 170, 13809, 23763, 2443, 1110, 3014]

这些输出，一旦转换为适当的框架tensor，就可以像本章前面看到的那样用作模型的输入。

#### Decoding

解码是另一种方式: 从词汇表索引，我们想要得到一个字符串。这可以通过decode()方法完成，如下所示:

In [13]:
decoded_string = tokenizer.decode(ids)
decoded_string

'Using a Transformer network is simple'

请注意，decode方法不仅将索引转换回标记，而且还将作为相同单词一部分的标记组合在一起，以生成可读的句子。当我们使用预测新文本的模型时(无论是从提示生成的文本，还是像翻译或摘要这样的序列到序列问题)，这种行为将非常有用。

到目前为止，您应该了解标记器可以处理的原子操作:标记化、转换为id以及将id转换回字符串。然而，我们只是触及了冰山一角。在下一节中，我们将探讨这种方法的局限性，并看看如何克服它们。